# **Modelo de Recomendação**
Nesse notebok será trabalhado no modelo de recomendação de produtos com dados de usuários de Olist, que foram selecionados e gerados a partir do notebook **analise_exploratoria_olist.ipnyb** (recomendo a leitura). Além de treinarmos o modelo para recomendação, vamos validar sua qualidade de previsão, testar dados absolutos e normalizados, e alguns modelos para entender qual seria a melhor opção para essa base de dados.

## **Resumo do Modelo**
AAAAAAAAAAAAAAA

In [17]:
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [10]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,order_id,customer_id,customer_unique_id,customer_city,customer_state,product_id,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,4
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,barreiras,BA,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,4
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,aa4383b373c6aca5d8797843e5594415,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,RN,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,santo andre,SP,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,5


In [19]:
#Selecionando características de produtos e usuários
produto_features = ["price", "freight_value", "product_category_name", 
                    "product_name_lenght", "product_description_lenght", "product_photos_qty",
                    "product_weight_g", "product_length_cm", "product_height_cm",
                    "product_width_cm"]
cliente_features = ["customer_city", "customer_state"]

**Gerando uma amostra negativa**
No dataset existe apenas dados de compras realizadas, como a intenção é desenvolver um modelo de classificação para realizar a recoemndação de produtos, será necessário criar uma amostra aleatória de compras não realizadas para auxiliar o modelo no entendimento dos dados.

In [13]:
positivos = dataset[["customer_unique_id", "product_id"]].drop_duplicates()
positivos['target'] = 1

clientes = dataset["customer_unique_id"].unique()
produtos = dataset["product_id"].unique()

In [14]:
#Gerando a amostra negativa
negativos = []

for _ in range(len(positivos)):
    c = random.choice(clientes)
    p = random.choice(produtos)
    if not ((positivos["customer_unique_id"] == c) & (positivos["product_id"]) == p).any():
        negativos.append((c, p, 0))
negativos = pd.DataFrame(negativos, columns=["customer_unique_id", "product_id", "target"])

In [21]:
#Juntando interacoes
interacoes = pd.concat([positivos, negativos])

#Juntando atributos de cliente e produtos
dados = interacoes.merge(dataset.drop_duplicates("customer_unique_id")[["customer_unique_id"] + cliente_features], 
                         on="customer_unique_id", how="left")
dados = dados.merge(dataset.drop_duplicates("product_id")[["product_id"] + produto_features], 
                    on="product_id", how="left")

**Preciso corrigir os dados vazios**

In [56]:
dados_teste = dados.copy()
dados_teste.dropna(inplace=True)

X = dados_teste[cliente_features + produto_features]
y = dados_teste['target']

num_features = list(dados_teste.select_dtypes("float").columns)
cat_features = list(dados_teste.select_dtypes("object").columns)
cat_features.remove("customer_unique_id")
cat_features.remove("product_id")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

modelo = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classicador', RandomForestClassifier(n_estimators=100))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

modelo.fit(X_train, y_train)

probas = modelo.predict_proba(X_test)[:, 1]

**Pré-processamento**

In [ ]:
X = dados[cliente_features + produto_features]
y = dados['target']

num_features = list(dados.select_dtypes("float").columns)
cat_features = list(dados.select_dtypes("object").columns)
cat_features.remove("customer_unique_id")
cat_features.remove("product_id")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

**Random Forest Classifier**

In [52]:
#Modelo
modelo = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classicador', RandomForestClassifier(n_estimators=100))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

modelo.fit(X_train, y_train)

In [ ]:
probas = modelo.predict_proba(X_test)[:, 1]

# **Recomendação**

In [58]:
cliente_id = dataset["customer_unique_id"][8]

In [59]:
produtos_comprados = dados[dados['customer_unique_id'] == cliente_id]['product_id'].unique()
todos_produtos = dados['product_id'].unique()

produtos_nao_comprados = [p for p in todos_produtos if p not in produtos_comprados]

In [60]:
# Pegamos as informações do cliente
cliente_info = dados[dados['customer_unique_id'] == cliente_id].iloc[0]

# Criar novas linhas combinando o cliente com os produtos não comprados
recomendacoes = []

for produto_id in produtos_nao_comprados:
    prod_info = dados[dados['product_id'] == produto_id].iloc[0]

    linha = {
        'customer_unique_id': cliente_id,
        'customer_state': cliente_info['customer_state'],
        'customer_city': cliente_info['customer_city'],
        'product_id': produto_id,
        'product_category_name': prod_info['product_category_name'],
        'price': prod_info['price'],
        'product_weight_g': prod_info['product_weight_g'],
        # adicione aqui as demais colunas usadas no modelo
    }

    recomendacoes.append(linha)

df_recomendacoes = pd.DataFrame(recomendacoes)


In [63]:
X.columns

Index(['customer_city', 'customer_state', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [61]:
X_recomendacao = df_recomendacoes[cliente_features + produto_features]
scores = modelo.predict_proba(X_recomendacao)[:, 1]

df_recomendacoes['score'] = scores


KeyError: "['freight_value', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_length_cm', 'product_height_cm', 'product_width_cm'] not in index"

In [ ]:
top_n = 5
top_recomendacoes = df_recomendacoes.sort_values(by='score', ascending=False).head(top_n)

print(top_recomendacoes[['product_id', 'product_category_name', 'score']])
